# `transformers` meets `bitsandbytes` for democratzing Large Language Models (LLMs) through 4bit quantization

<center>
<img src="https://github.com/huggingface/blog/blob/main/assets/96_hf_bitsandbytes_integration/Thumbnail_blue.png?raw=true" alt="drawing" width="700" class="center"/>
</center>

Welcome to this notebook that goes through the recent `bitsandbytes` integration that includes the work from XXX that introduces no performance degradation 4bit quantization techniques, for democratizing LLMs inference and training.

In this notebook, we will learn together how to load a large model in 4bit (`gpt-neo-x-20b`) and train it using Google Colab and PEFT library from Hugging Face 🤗.

[In the general usage notebook](https://colab.research.google.com/drive/1ge2F1QSK8Q7h0hn3YKuBCOAS0bK8E0wf?usp=sharing), you can learn how to propely load a model in 4bit with all its variants.

If you liked the previous work for integrating [*LLM.int8*](https://arxiv.org/abs/2208.07339), you can have a look at the [introduction blogpost](https://huggingface.co/blog/hf-bitsandbytes-integration) to lean more about that quantization method.


In [1]:
import numpy as np
from scipy.special import softmax
import pdb
import pandas as pd
import math
from typing import List
import random
import argparse
import torch


def sent_scoring(model_tokenizer, text, cuda, score_type="loss", output_attentions=False, length_normalize=False):
    model = model_tokenizer[0]
    tokenizer = model_tokenizer[1]
    assert model is not None
    assert tokenizer is not None
    encoded_text = tokenizer.encode(text)
    input_ids = torch.tensor(encoded_text).unsqueeze(0)
    if cuda:
        input_ids = input_ids.to('cuda')
    with torch.no_grad():
        outputs = model(input_ids, labels=input_ids, output_attentions=output_attentions)
    loss, logits = outputs[:2]

    sentence_prob = loss.item()
    if score_type == "prob":
        if length_normalize:
            mult = 2
        else:
            mult = len(encoded_text)

        sentence_prob = math.exp(-1.0 * loss * (mult - 1))

    if output_attentions:
        attn = outputs["attentions"]
        return sentence_prob, attn, input_ids

    return sentence_prob

def confusion_matrix(P_forward_1, P_forward_2, P_backward_1, P_backward_2):
    correct_forward = len(np.where(np.array(P_forward_1) >= 0.5)[0]) + len(np.where(np.array(P_forward_2) >=0.5)[0])
    wrong_forward = len(P_forward_1) + len(P_forward_2) - correct_forward

    correct_backward = len(np.where(np.array(P_backward_1) >= 0.5)[0]) + len(np.where(np.array(P_backward_2) >=0.5)[0])
    wrong_backward = len(P_backward_1) + len(P_backward_2) - correct_backward

    print("correct forward", correct_forward, "wrong forward", wrong_forward, "correct backward", correct_backward, "wrong_backward", wrong_backward)

    results = {
        "correct_forward": correct_forward,
        "wrong_forward": wrong_forward,
        "correct_backward": correct_backward,
        "wrong_backward": wrong_backward
    }

    return results

from tqdm import tqdm

def evaluate_model(model, tokenizer, test_set, middle_phrase="", use_prefix=0, verbose=True, score_type="prob", use_cuda=False, return_acc=False, total = 1094) -> tuple:
    preds = []
    labels = []
    x_1 = []
    x_2 = []
    y_1 = []
    y_2 = []
    P_x_1 = []
    P_x_2 = []
    P_y_1 = []
    P_y_2 = []
    P_x_1_y_1 = []
    P_x_1_y_2 = []
    P_x_2_y_1 = []
    P_x_2_y_2 = []
    P_x_1_correct = []
    P_x_2_correct = []
    P_y_1_correct = []
    P_y_2_correct = []
    correct = 0

    for i, metaphor_data in tqdm(enumerate(test_set), total = total):
        ctx, p1, p2 = metaphor_data["startphrase"], metaphor_data["ending1"], metaphor_data["ending2"]
        labels.append(int(metaphor_data["labels"]))
        if use_prefix > 0:
            prefix_prompt = select_prefix_prompts(prompt_file, use_prefix) if use_prefix else ""
        else:
            prefix_prompt = ""

        sent1 = prefix_prompt + ctx + ". " + middle_phrase + p1 + "."
        sent2 = prefix_prompt + ctx + ". " + middle_phrase + p2 + "."

        score1 = sent_scoring((model, tokenizer), sent1, use_cuda, score_type=score_type)
        score2 = sent_scoring((model, tokenizer), sent2, use_cuda, score_type=score_type)

        if score_type == "loss":
            pred = 0 if score1 < score2 else 1
        else:
            pred = 1 if score1 < score2 else 0

        pred_sent = sent1 if pred == 0 else sent2

        if i % 2 == 0:
            x_1.append(ctx)
            x_1_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_1.append(x_1_score)
            y_1.append(p1)
            y_2.append(p2)
            y1_score = sent_scoring((model, tokenizer), p1 + ".", use_cuda, score_type=score_type)
            y2_score = sent_scoring((model, tokenizer), p2 + ".", use_cuda, score_type=score_type)
            P_y_1.append(y1_score)
            P_y_2.append(y2_score)

            P_x_1_y_1.append(score1)
            P_x_1_y_2.append(score2)
            P_x_1_correct.append(score1/(score1 + score2))

        else:
            x_2.append(ctx)
            x_2_score = sent_scoring((model, tokenizer), ctx + ".", use_cuda, score_type=score_type)
            P_x_2.append(x_2_score)
            P_x_2_y_1.append(score1)
            P_x_2_y_2.append(score2)
            P_x_2_correct.append(score2/(score1 + score2))

            P_y_1_correct.append(P_x_1_y_1[-1]/(P_x_1_y_1[-1] + score1))
            P_y_2_correct.append(score2/(P_x_1_y_2[-1] + score2))

        if verbose:
            print(f"Q: {ctx}: 1. {p1} 2. {p2}")
            print(f"model says '{pred_sent}' is more likely")
            print("\n")
        if pred == metaphor_data["labels"]:
            correct += 1
        preds.append(pred)

    cols = {"x_1": x_1, "x_2": x_2, "y_1": y_1, "y_2": y_2, "P(x_1)": P_x_1, "P(x_2)": P_x_2, "P(y_1)": P_y_1, "P(y_2)": P_y_2,
        "P(x_1, y_1)": P_x_1_y_1, "P(x_1, y_2)": P_x_1_y_2, "P(x_2, y_1)": P_x_2_y_1, "P(x_2, y_2)": P_x_2_y_2,
        "P(y_1|x_1)": P_x_1_correct, "P(y_2|x_2)": P_x_2_correct, "P(x_1|y_1)": P_y_1_correct, "P(x_2|y_2)": P_y_2_correct}
    out_df = pd.DataFrame(cols)

    if return_acc:
        return correct/len(preds), out_df, preds, labels

    return out_df, preds, labels

def compute_stats(total_df: pd.DataFrame, all_preds: List, all_labels: List) -> None:
    print("overall accuracy: ")
    accuracyy = len(np.where(np.array(all_preds) == np.array(all_labels))[0])/len(all_labels)
    print(accuracyy)
    print("confusion matrix: ")
    matrix_dic = confusion_matrix(list(total_df["P(y_1|x_1)"]), list(total_df["P(y_2|x_2)"]), list(total_df["P(x_1|y_1)"]), list(total_df["P(x_2|y_2)"]))

    return accuracyy, matrix_dic


In [2]:
!pip uninstall datasets -y
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 33.6 MB/s eta 0:00:00


In [2]:
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 10.1 MB/s eta 0:00:00


In [3]:
!pip install -q -U transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.0/302.0 kB 34.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 51.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 49.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 16.8 MB/s eta 0:00:00


In [4]:
!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 3.0 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 5.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.1 MB/s eta 0:00:00


In [5]:
from datasets import load_dataset

First let's load the model we are going to use - GPT-neo-x-20B! Note that the model itself is around 40GB in half precision

In [7]:
!pip install einops

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 851.0 kB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

#model_id = "microsoft/phi-1_5"
model_id = "mistralai/Mistral-7B-v0.1"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
subset_test_dataset[0]

{'startphrase': 'The girl had the flightiness of a sparrow',
 'ending1': 'The girl was very fickle.',
 'ending2': 'The girl was very stable.',
 'labels': 0,
 'valid': 1}

In [21]:
subset_test_dataset = dataset['validation'].select(range(10))
out_df, preds, labels = evaluate_model(model, tokenizer, subset_test_dataset, verbose = True, total = 10)

 10%|█         | 1/10 [00:01<00:14,  1.64s/it]

Q: The girl had the flightiness of a sparrow: 1. The girl was very fickle. 2. The girl was very stable.
model says 'The girl had the flightiness of a sparrow. The girl was very fickle..' is more likely




 20%|██        | 2/10 [00:02<00:08,  1.04s/it]

Q: The girl had the flightiness of a rock: 1. The girl was very fickle. 2. The girl was very stable.
model says 'The girl had the flightiness of a rock. The girl was very fickle..' is more likely




 30%|███       | 3/10 [00:03<00:07,  1.03s/it]

...

Q: Shopping for groceries is a scavenger hunt with a list created by a lunatic: 1. Shopping for groceries is a fun, rewarding chore 2. Shopping for groceries is a crazy, nearly impossible chore
model says 'Shopping for groceries is a scavenger hunt with a list created by a lunatic. Shopping for groceries is a fun, rewarding chore.' is more likely




 90%|█████████ | 9/10 [00:10<00:01,  1.21s/it]

Q: War is an amputation on the wrong limb: 1. War is the wrong solution to a problem 2. War is a necessary solution
model says 'War is an amputation on the wrong limb. War is the wrong solution to a problem.' is more likely




100%|██████████| 10/10 [00:11<00:00,  1.10s/it]

Q: War is an amputation to save your life: 1. War is the wrong solution to a problem 2. War is a necessary solution
model says 'War is an amputation to save your life. War is a necessary solution.' is more likely




In [19]:
# 1. Tokenize the input string
input_string = "'The music was loud like a whisper' is a metaphor. In other words, the metaphor means"
input_tokens = tokenizer.encode(input_string, return_tensors='pt')

# 2. Convert tokenized input to tensor
# Assuming you're using a GPU, move the tensor to the GPU
input_tokens = input_tokens.to('cuda:0')  # Replace 'cuda:0' with the appropriate device if not using GPU.

# 3. Pass the tensor to the model
with torch.no_grad():
    output_tokens = model.generate(input_tokens, max_new_tokens=40)

# 4. Decode the model's output to get the prediction
output_string = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(output_string)


'The music was loud like a whisper' is a metaphor. In other words, the metaphor means that the music was quiet and gentle, like a whisper.

Now, let's talk about how we can use similes and metaphors in our writing. One way is to use them to make


In [28]:
# 1. Tokenize the input string
input_string = """ Choose one option that explains the metaphor

The leaves were as green as dirt:
1. The leaves were very green
2. The leaves were brown and not green at all.

Answer:
"""
input_tokens = tokenizer.encode(input_string, return_tensors='pt')

# 2. Convert tokenized input to tensor
# Assuming you're using a GPU, move the tensor to the GPU
input_tokens = input_tokens.to('cuda:0')  # Replace 'cuda:0' with the appropriate device if not using GPU.

# 3. Pass the tensor to the model
with torch.no_grad():
    output_tokens = model.generate(input_tokens, max_new_tokens=40)

# 4. Decode the model's output to get the prediction
output_string = tokenizer.decode(output_tokens[0], skip_special_tokens=True)

print(output_string)


 Choose one option that explains the metaphor

The leaves were as green as dirt: 
1. The leaves were very green 
2. The leaves were brown and not green at all.

Answer: 
1. The leaves were very green.

Exercise 2:

Fill in the blank with the correct simile:

Her smile was as bright as the __.

Answer


Then we have to apply some preprocessing to the model to prepare it for training. For that use the `prepare_model_for_kbit_training` method from PEFT.

## Metaphor Probabilities (zero-shot) for the pretrained model

In [20]:
dataset = load_dataset("nightingal3/fig-qa")

Extracting data files:   0%|          | 0/3 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Generating test split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset['validation']

Dataset({
    features: ['startphrase', 'ending1', 'ending2', 'labels', 'valid'],
    num_rows: 1094
})

In [53]:
subset_test_dataset = dataset['validation'].select(range(500))

In [ ]:
out_df, preds, labels = evaluate_model(model, tokenizer, subset_test_dataset, verbose = False, total = 500)

100%|██████████| 500/500 [04:48<00:00,  1.73it/s]


In [ ]:
zero_shot_accuracy, conf_matrix_zero_shot =  compute_stats(out_df, preds, labels)

overall accuracy: 
0.606
confusion matrix: 
correct forward 303 wrong forward 197 correct backward 284 wrong_backward 216


In [ ]:
import json

model_id_string = model_id.replace("/", "-")

# Saving DataFrame to CSV
out_df.to_csv(f'output_df_{model_id_string}.csv', sep="\t", index=False)

# Saving other data as JSON
data_to_save = {
    "preds": preds,
    "labels": labels,
    "zero_shot_accuracy": zero_shot_accuracy,
    "conf_matrix_zero_shot": conf_matrix_zero_shot
}

with open(f'output_data_{model_id_string}.json', 'w') as file:
    json.dump(data_to_save, file)

import pickle

data_to_save_pick = {
    "out_df": out_df,
    "preds": preds,
    "labels": labels,
    "zero_shot_accuracy": zero_shot_accuracy,
    "conf_matrix_zero_shot": conf_matrix_zero_shot
}

with open(f'pickle_output_data_{model_id_string}.pkl', 'wb') as file:
    pickle.dump(data_to_save_pick, file)



## Finetuning

In [ ]:
# from peft import prepare_model_for_kbit_training

# model.gradient_checkpointing_enable()
# model = prepare_model_for_kbit_training(model)

In [ ]:
def print_modules(model, prefix=''):
    for name, module in model.named_children():
        full_name = f"{prefix}.{name}" if prefix else name
        print(full_name)
        print_modules(module, full_name)

print_modules(model)

In [31]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [32]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    #target_modules=["query_key_value"],
    target_modules=["Wqkv"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model_peft = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 1572864 || all params: 815863808 || trainable%: 0.1927851173905731


Let's load a common dataset, english quotes, to fine tune our model on famous quotes.

In [33]:
def map_concatenation_and_tokenization(samples):
    concatenated_phrases = []
    input_ids_list = []
    attention_mask_list = []

    for i in range(len(samples['startphrase'])):
        # Construct the concatenated phrase in the desired format
        concatenated_phrase = (
            "Choose one option that explains the metaphor\n\n" +
            samples['startphrase'][i] + ":\n" +
            "1. " + samples['ending1'][i] + "\n" +
            "2. " + samples['ending2'][i] + "\n\n" +
            "Answer: the correct interpretation is " + str(samples['labels'][i] + 1)  # Since labels are 0-based and we want 1 or 2
        )
        concatenated_phrases.append(concatenated_phrase)

        # Tokenize the concatenated_phrase
        tokens = tokenizer(concatenated_phrase, truncation=True, max_length=512, return_tensors='pt')
        input_ids_list.append(tokens['input_ids'][0].tolist())
        attention_mask_list.append(tokens['attention_mask'][0].tolist())

    return {
        'concatenated_phrase': concatenated_phrases,
        'input_ids': input_ids_list,
        'attention_mask': attention_mask_list
    }

# Apply the mapping function
data = load_dataset("nightingal3/fig-qa")
data = data.map(map_concatenation_and_tokenization, batched=True)


Map:   0%|          | 0/9674 [00:00<?, ? examples/s]

Map:   0%|          | 0/1094 [00:00<?, ? examples/s]

Map:   0%|          | 0/1146 [00:00<?, ? examples/s]

Run the cell below to run the training! For the sake of the demo, we just ran it for few steps just to showcase how to use this integration with existing tools on the HF ecosystem.

In [ ]:
model_peft.to(device)

In [36]:
import transformers
from transformers import EarlyStoppingCallback

# Assuming you've already loaded the tokenizer and set pad_token.
tokenizer.pad_token = tokenizer.eos_token

# TrainingArguments for the trainer
training_args = transformers.TrainingArguments(
    per_device_train_batch_size=15,
    gradient_accumulation_steps=4,
    warmup_steps=2,
    max_steps=1000,
    learning_rate= 2e-4,
    fp16=True,
    logging_steps=1,
    output_dir="outputs",
    optim="paged_adamw_8bit",

    # Add these arguments for evaluation and early stopping
    evaluation_strategy="steps",  # or "epoch"
    eval_steps=10,  # Evaluation and Save happens every 10 steps
    save_strategy="steps",  # Save model checkpoint every 10 steps
    save_steps=10,
    load_best_model_at_end=True,  # Load the best model at the end of training (based on metric)
    metric_for_best_model="loss",  # Use loss to determine the best model
    lr_scheduler_type="cosine",  # Use cosine annealing
    #lr_scheduler_num_cycles=0.5,  # Determines the number of cosine annealing cycles

)

# Initialize Trainer
trainer = transformers.Trainer(
    model=model_peft,
    train_dataset=data["train"],
    eval_dataset=data["validation"].select(range(30)),
    args=training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=10)],  # Stop training once the metric doesn't improve for 3 evals
)

model_peft.config.use_cache = False  # Silence the warnings. Please re-enable for inference!
trainer.train()


`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.


Step,Training Loss,Validation Loss
10,1.354300,1.162498
20,1.172200,1.126146
30,1.174600,1.100520
40,1.166400,1.086376
50,1.105000,1.075024
60,1.126200,1.071610
70,1.084600,1.062374
80,1.003300,1.055523
90,1.035400,1.046598
100,1.083600,1.043027


`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it might lead to unexpected results.
`attention_mask` is not supported during training. Using it 

TrainOutput(global_step=460, training_loss=1.0569494463827298, metrics={'train_runtime': 1442.836, 'train_samples_per_second': 41.585, 'train_steps_per_second': 0.693, 'total_flos': 1.3052106411921408e+16, 'train_loss': 1.0569494463827298, 'epoch': 2.85})

In [37]:
model_to_save = trainer.model.module if hasattr(trainer.model, 'module') else trainer.model  # Take care of distributed/parallel training
model_to_save.save_pretrained("outputs")

In [38]:
lora_config = LoraConfig.from_pretrained('outputs')
model_finetuned = get_peft_model(model_peft, lora_config)

In [ ]:
device = "cuda:0"

model_finetuned.to(device)

In [48]:
text = """ Choose one option that explains the metaphor

The leaves were as green as dirt:
1. The leaves were very green
2. The leaves were brown and not green at all.

Answer:
"""

device = "cuda:0"

model_finetuned.to(device)

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model_finetuned.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

 Choose one option that explains the metaphor

The leaves were as green as dirt: 
1. The leaves were very green 
2. The leaves were brown and not green at all.

Answer: 
1. The leaves were very green.

Exercise 2:

Fill in the blank


In [50]:
text = """ Choose one option that explains the metaphor

It was as peaceful as a battlefield.:
1. It was very peaceful.
2. It was full of conflict and danger, not peace.

Answer:
"""

device = "cuda:0"

model_finetuned.to(device)

inputs = tokenizer(text, return_tensors="pt").to(device)
outputs = model_finetuned.generate(**inputs, max_new_tokens=20)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

 Choose one option that explains the metaphor

It was as peaceful as a battlefield.: 
1. It was very peaceful. 
2. It was full of conflict and danger, not peace.

Answer: 
1. It was very peaceful.
2. It was full of conflict and danger, not peace


In [54]:
out_df_finetuned, preds_finetuned, labels_finetuned = evaluate_model(model_finetuned, tokenizer, subset_test_dataset, verbose = False, total = 500)

100%|██████████| 500/500 [03:45<00:00,  2.21it/s]


In [55]:
zero_shot_accuracy_finetuned, conf_matrix_zero_shot_finetuned =  compute_stats(out_df_finetuned, preds_finetuned, labels_finetuned)

overall accuracy: 
0.608
confusion matrix: 
correct forward 304 wrong forward 196 correct backward 283 wrong_backward 217


In [ ]:
import json

model_id_string = model_id.replace("/", "-")

# Saving DataFrame to CSV
out_df_finetuned.to_csv(f'output_df_{model_id_string}_finetuned.csv', sep="\t", index=False)

# Saving other data as JSON
data_to_save = {
    "preds": preds_finetuned,
    "labels": labels_finetuned,
    "zero_shot_accuracy": zero_shot_accuracy_finetuned,
    "conf_matrix_zero_shot": conf_matrix_zero_shot_finetuned
}

with open(f'output_data_{model_id_string}_finetuned.json', 'w') as file:
    json.dump(data_to_save, file)

import pickle

data_to_save_pick = {
    "out_df": out_df_finetuned,
    "preds": preds_finetuned,
    "labels": labels_finetuned,
    "zero_shot_accuracy": zero_shot_accuracy_finetuned,
    "conf_matrix_zero_shot": conf_matrix_zero_shot_finetuned
}

with open(f'pickle_output_data_{model_id_string}_finetuned.pkl', 'wb') as file:
    pickle.dump(data_to_save_pick, file)

